In [0]:
!pip install gpustat
!pip install --upgrade keras
!pip install --upgrade keras_preprocessing
!gpustat

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)
Requirement already up-to-date: keras_preprocessing in /usr/local/lib/python3.6/dist-packages (1.1.0)
ce4fa6a3e2a2             Fri Nov 22 18:22:41 2019  418.67
[0] Tesla P100-PCIE-16GB | 37'C,   0 % |     0 / 16280 MB |


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

drive.mount('/content/gdrive')
keras.backend.tensorflow_backend._get_available_gpus()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
data_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_data/train/'
label_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_label/train/'

In [0]:
BATCH_SIZE = 4
GENERATE_SEED = np.random.randint(0, 1024)
VALIDATION_SPLIT = 0.2

maskgen_args = dict(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=VALIDATION_SPLIT
)

imagegen_args = dict(
    samplewise_center=True,
    samplewise_std_normalization=True,
    **maskgen_args
)

image_datagen = ImageDataGenerator(**imagegen_args)
mask_datagen = ImageDataGenerator(**maskgen_args)

training_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='training'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='training'
    )
)

validation_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='validation'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='validation'
    )
)

Found 9672 images belonging to 1 classes.
Found 9672 images belonging to 1 classes.
Found 2418 images belonging to 1 classes.
Found 2418 images belonging to 1 classes.


In [0]:
# Reference: https://github.com/zhixuhao/unet/blob/master/model.py

def unet(input_size=(512,512,3)):
    inputs = Input(input_size)
    noisy_inputs = GaussianNoise(stddev=0.05)(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(noisy_inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    return model

In [0]:
unet_model = unet()
unet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [15]:
unet_model.fit_generator(
    training_data_generator, 
    validation_data=validation_data_generator,
    epochs=6,
    steps_per_epoch=4096,
    validation_steps=1024,
    validation_freq=1,
    callbacks=[keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/data_source/food_segmentation/unet/model_checkpoint_{epoch:02d}.hdf5")]
)

Epoch 1/6
4096/4096 [==============================] - 2799s 683ms/step - loss: 0.4909 - accuracy: 0.7704 - val_loss: 0.3262 - val_accuracy: 0.8105
Epoch 2/6
4096/4096 [==============================] - 2800s 684ms/step - loss: 0.4142 - accuracy: 0.8312 - val_loss: 0.7101 - val_accuracy: 0.8463
Epoch 3/6
4096/4096 [==============================] - 2797s 683ms/step - loss: 0.3814 - accuracy: 0.8434 - val_loss: 0.3320 - val_accuracy: 0.8552
Epoch 4/6
4096/4096 [==============================] - 2807s 685ms/step - loss: 0.3586 - accuracy: 0.8512 - val_loss: 0.2264 - val_accuracy: 0.8601
Epoch 5/6
4096/4096 [==============================] - 2784s 680ms/step - loss: 0.3436 - accuracy: 0.8563 - val_loss: 0.2851 - val_accuracy: 0.8598
Epoch 6/6
4096/4096 [==============================] - 2772s 677ms/step - loss: 0.3343 - accuracy: 0.8594 - val_loss: 0.1801 - val_accuracy: 0.8646
